In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [23]:
from pandas_profiling import ProfileReport

In [9]:
df = pd.concat([
    pd.read_csv('./datasets/datasets/dataset-part1.csv'), 
    pd.read_csv('./datasets/datasets/dataset-part2.csv')
])

In [10]:
df.shape

(12207873, 50)

In [33]:
## generar profile report
profile = ProfileReport(df.sample(1000000), minimal=True)

In [34]:
## guardar profile como .html
profile.to_file("./reports/profile_report.html")